# Importar Librerias

In [ ]:
# ----------- 1. Create DGL Graph -------------- #
data_path = "datasets/DGL_Graph/MYCODEDiGraph/"
# data_path = "datasets/DGL_Graph/MYCODEDiGraphDegree/"
gnn = GNN(debug=False)
gnn.load_dataset( data_path, force_reload=True)
gnn.split_dataset_edges(0.6)

labels = gnn.dgl_graph.edata["Relationship"].float()

In [ ]:
# ----------- 2. Run PageRank for Graph -------------- #
N = gnn.dgl_graph.number_of_nodes()
DAMP = 0.85
K = 10


def compute_pagerank(g):
    g.ndata["pv"] = torch.ones(N) / N
    degrees = g.out_degrees(g.nodes()).type(torch.float32)
    for k in range(K):
        g.ndata["pv"] = g.ndata["pv"] / degrees
        g.update_all(
            message_func=fn.copy_u(u="pv", out="m"),
            reduce_func=fn.sum(msg="m", out="pv"),
        )
        g.ndata["pv"]  = (1 - DAMP) / N + DAMP * g.ndata["pv"]
    g.ndata["pv"]  = g.ndata["pv"].unsqueeze(1)
    return g.ndata["pv"]


pv = compute_pagerank(gnn.dgl_graph)
gnn.dgl_graph.ndata["h"] = pv  # Inicializa características del nodo

print(gnn.dgl_graph.ndata["pv"])



In [ ]:


# Definir Dataloader y Sampler Training
sampler_train = dgl.dataloading.NeighborSampler([2, 2])


# Transformar el sampler para predicción de aristas
sampler_train = dgl.dataloading.as_edge_prediction_sampler(
    sampler_train, 
    # exclude='reverse_id',  # Excluir aristas inversas
    # reverse_eids=reverse_eids,  # Especificar las aristas inversas
    # negative_sampler=dgl.dataloading.negative_sampler.Uniform(5)  # Sampler negativo con 5 muestras negativas
)

train_dataloader = dgl.dataloading.DataLoader(
    # The following arguments are specific to DGL's DataLoader.
    gnn.dgl_graph,              # The graph
    train_nids,         # The node IDs to iterate over in minibatches
    sampler_train,            # The neighbor sampler
    # The following arguments are inherited from PyTorch DataLoader.
    batch_size=1000,    # Batch size
    shuffle=True,       # Whether to shuffle the nodes for every epoch
    drop_last=False,    # Whether to drop the last incomplete batch
    # num_workers=0       # Number of sampler processes
)



# Definir Dataloader y Sampler Evaluación ------------------------------------------------------------------
sampler_test = dgl.dataloading.NeighborSampler([2,2])


# Transformar el sampler para predicción de aristas
sampler_test = dgl.dataloading.as_edge_prediction_sampler(
    sampler_test, 
    # exclude='reverse_id',  # Excluir aristas inversas
    # reverse_eids=reverse_eids,  # Especificar las aristas inversas
    # negative_sampler=dgl.dataloading.negative_sampler.Uniform(5)  # Sampler negativo con 5 muestras negativas
)

test_dataloader = dgl.dataloading.DataLoader(
    # The following arguments are specific to DGL's DataLoader.
    gnn.dgl_graph,              # The graph
    test_nids,         # The node IDs to iterate over in minibatches
    sampler_test,            # The neighbor sampler
    # The following arguments are inherited from PyTorch DataLoader.
    batch_size=1000,    # Batch size
    shuffle=True,       # Whether to shuffle the nodes for every epoch
    drop_last=False,    # Whether to drop the last incomplete batch
    # num_workers=0       # Number of sampler processes
)



In [ ]:


# # model = GraphSAGE(num_features_in, num_feat_hidden,num_feat_out)
model = ModelSAGESample(gnn.dgl_graph.ndata["feat"].size(1), 100, 25)
pred = MLPPredictor(25,1)

# ----------- 3. set up loss and optimizer -------------- #

optimizer = torch.optim.Adam( pred.parameters(), lr=0.05)

# ----------- 4. training -------------------------------- #
acc_val = []
acc_train = []

train_error = []
train_error_per_epoch = []
val_error_per_epoch = []
train_edges_id = []

# Inicializar listas para guardar logits y labels del entrenamiento
all_logits_train = []
all_labels_train = []

# Inicializar listas para guardar logits y labels de la validacion
all_logits_val = []
all_labels_val = []
# Almacenar la importancia de características por epoch
feature_importances = []
for epoch in range(5):
    model.train()
    val_loss_epoch=0
    num_batches = 0

    with tqdm.tqdm(train_dataloader) as tq:
        for step, (input_nodes, output_graph, mfgs) in enumerate(tq):
    
            mfgs = [graph for graph in mfgs]
            edge_ids = output_graph.edata[dgl.EID]
            inputs = mfgs[-1].srcdata['pv']

            m = compute_pagerank(output_graph)
            # Hacer que los inputs requieran gradiente
            inputs.requires_grad_(True)
            # Embeddings de nodos
            print(inputs)
            logits =  pred(output_graph, m)
            
            
#             train_edges_id.append(edge_ids)

#             # Usar los IDs de los bordes para obtener las etiquetas reales
#             labels_batch_output = gnn.dgl_graph.edata['Relationship'][edge_ids].float()

#             all_logits_train.append(logits.detach().cpu().numpy())
#             all_labels_train.append(labels_batch_output.cpu().numpy())
#             # print("LOGITS",logits,logits.shape)
#             # print("LABEL",mfgs[-1].edata['Relationship'].shape )

#             # Calculo pérdida --------------------------------------------------------
#             train_loss = F.binary_cross_entropy_with_logits(logits, labels_batch_output)
#             train_error.append(train_loss.item())


#             val_loss_epoch += train_loss.item()
#             num_batches += 1
            
#             # backward
#             optimizer.zero_grad()
#             # train_loss.backward()
#             train_loss.backward()

#                         # Capturar los gradientes con respecto a los inputs (características)
#             grad = inputs.grad  # Este es el gradiente de la pérdida con respecto a los inputs
#             feature_importance_batch = grad.abs().mean(dim=0).cpu().numpy()  # Promediar sobre los nodos y las dimensiones de características
#             feature_importances.append(feature_importance_batch)
#             optimizer.step()

#     train_error_per_epoch.append(val_loss_epoch / num_batches)
        
#     # Validación
#     model.eval()
#     val_loss_epoch = 0
#     num_batches = 0
#     with torch.no_grad():
#         with tqdm.tqdm(test_dataloader) as tq:
#             for step, (input_nodes, output_graph, mfgs) in enumerate(tq):
#                 mfgs = [graph for graph in mfgs]
#                 edge_ids = output_graph.edata[dgl.EID]
#                 inputs = mfgs[0].srcdata['feat']
                
                
#                 # Forward para validación
#                 h_val = model(mfgs, inputs)
#                 logits_val = pred(output_graph, h_val)

#                 # Guardar logits de validacion
#                 all_logits_val.append(logits_val.detach().cpu().numpy())

#                 # Obtener las etiuetas reales
#                 labels_batch_output_val = gnn.dgl_graph.edata['Relationship'][edge_ids].float()
#                 all_labels_val.append(labels_batch_output_val.cpu().numpy())
            
#                 # Calcular pérdida de validación
#                 val_loss_batch = F.binary_cross_entropy_with_logits(logits_val, labels_batch_output_val)
#                 val_loss_epoch += val_loss_batch.item()
#                 num_batches += 1

#     # Almacenar el promedio de la pérdida de validación por epoch
#     val_error_per_epoch.append(val_loss_epoch / num_batches)


#     if epoch % 1 == 0:
#         print('In epoch {}, train loss: {},'.format( epoch,train_loss))



# # ----------- 5. Definir threashold -------------- 
# # Concatenar todos los logits y etiquetas al final del entrenamiento
# all_logits_train = np.concatenate(all_logits_train, axis=0)
# all_labels_train = np.concatenate(all_labels_train, axis=0)
# optimal_threshold = plot_roc_curve(all_labels_train, all_logits_train)
# print(f"Optimal Threshold: {optimal_threshold}")
          

# # ----------- 6. plot the loss and accuracy de training-------------- #
# # FIXME: Ess asi por meintras para q no s ecaiga la funcion
# print("Train Error",train_error)
# # acc_train = train_error
# # val_error = train_error
# # acc_val = train_error
# # model_complexity = [i for i in range(len(train_error))]


# # Crear gráfico del error de entrenamiento
# plt.figure()  # Crear una nueva figura
# plt.plot([i for i in range(len(train_error))],train_error, label='Train Error', color='blue')  # Error de entrenamiento en azul
# plt.xlabel('Mini-Batch')
# plt.ylabel('Error')
# plt.title('Training Error over Epochs')
# plt.legend()
# plt.grid(True)
# plt.show()  # Mostrar el gráfico de error de entrenamiento


# # ----------- 6. Visualización de la Importancia de Características -------------- #
# # Convertir la lista de importancias en un array para análisis
# # feature_importances = np.array(feature_importances)

# # Calcular la importancia promedio a lo largo de todos los batches y epochs
# # avg_feature_importance = feature_importances.mean(axis=0)

# # plt.figure()
# # plt.bar(range(len(avg_feature_importance)), avg_feature_importance)
# # plt.xlabel('Feature Index')
# # plt.ylabel('Average Importance')
# # plt.title('Average Feature Importance over Training')
# # # plt.show()

